# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f533cae1040>
├── 'a' --> tensor([[ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488]])
└── 'x' --> <FastTreeValue 0x7f533cb30670>
    └── 'c' --> tensor([[-0.9446,  0.9659,  2.3515,  0.8332],
                        [ 0.8201, -0.2388,  0.7523, -0.7483],
                        [ 0.2555, -0.8896,  1.2533,  1.1000]])

In [4]:
t.a

tensor([[ 0.3523, -1.3877, -1.4078],
        [ 0.7722,  1.4825, -0.1488]])

In [5]:
%timeit t.a

59.1 ns ± 0.221 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f533cae1040>
├── 'a' --> tensor([[ 0.1454,  0.3464,  0.4325],
│                   [-1.6325,  1.3501,  0.1428]])
└── 'x' --> <FastTreeValue 0x7f533cb30670>
    └── 'c' --> tensor([[-0.9446,  0.9659,  2.3515,  0.8332],
                        [ 0.8201, -0.2388,  0.7523, -0.7483],
                        [ 0.2555, -0.8896,  1.2533,  1.1000]])

In [7]:
%timeit t.a = new_value

68.4 ns ± 0.396 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.3523, -1.3877, -1.4078],
               [ 0.7722,  1.4825, -0.1488]]),
    x: Batch(
           c: tensor([[-0.9446,  0.9659,  2.3515,  0.8332],
                      [ 0.8201, -0.2388,  0.7523, -0.7483],
                      [ 0.2555, -0.8896,  1.2533,  1.1000]]),
       ),
)

In [10]:
b.a

tensor([[ 0.3523, -1.3877, -1.4078],
        [ 0.7722,  1.4825, -0.1488]])

In [11]:
%timeit b.a

51.2 ns ± 0.126 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2380, -1.7456, -1.3836],
               [ 0.4061, -0.1303, -1.1490]]),
    x: Batch(
           c: tensor([[-0.9446,  0.9659,  2.3515,  0.8332],
                      [ 0.8201, -0.2388,  0.7523, -0.7483],
                      [ 0.2555, -0.8896,  1.2533,  1.1000]]),
       ),
)

In [13]:
%timeit b.a = new_value

478 ns ± 0.142 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

846 ns ± 0.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 57.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 949 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f529845ff10>
├── 'a' --> tensor([[[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]],
│           
│                   [[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]],
│           
│                   [[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]],
│           
│                   [[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]],
│           
│                   [[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]],
│           
│                   [[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]],
│           
│                   [[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]],
│           
│                   [[ 0.3523, -1.3877, -1.4078],
│                    [ 0.7722,  1.4825, -0.1488]]])
└── 'x' --> <FastTreeValue 0x7f529845fb80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 44.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5298467880>
├── 'a' --> tensor([[ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488],
│                   [ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488],
│                   [ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488],
│                   [ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488],
│                   [ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488],
│                   [ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488],
│                   [ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488],
│                   [ 0.3523, -1.3877, -1.4078],
│                   [ 0.7722,  1.4825, -0.1488]])
└── 'x' --> <FastTreeValue 0x7f52964d4f70>
    └── 'c' --> tensor([[-0.9446,  0.9659,  2.3515,  0.8332],
                        [ 0.8201, -0.2388,  0.7523, -0.7483],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 51.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.1 µs ± 147 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.9446,  0.9659,  2.3515,  0.8332],
                       [ 0.8201, -0.2388,  0.7523, -0.7483],
                       [ 0.2555, -0.8896,  1.2533,  1.1000]],
              
                      [[-0.9446,  0.9659,  2.3515,  0.8332],
                       [ 0.8201, -0.2388,  0.7523, -0.7483],
                       [ 0.2555, -0.8896,  1.2533,  1.1000]],
              
                      [[-0.9446,  0.9659,  2.3515,  0.8332],
                       [ 0.8201, -0.2388,  0.7523, -0.7483],
                       [ 0.2555, -0.8896,  1.2533,  1.1000]],
              
                      [[-0.9446,  0.9659,  2.3515,  0.8332],
                       [ 0.8201, -0.2388,  0.7523, -0.7483],
                       [ 0.2555, -0.8896,  1.2533,  1.1000]],
              
                      [[-0.9446,  0.9659,  2.3515,  0.8332],
                       [ 0.8201, -0.2388,  0.7523, -0.7483],
                       [ 0.2555, -0.8896,  1.2533,  1.1000]],

In [26]:
%timeit Batch.stack(batches)

79.6 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.9446,  0.9659,  2.3515,  0.8332],
                      [ 0.8201, -0.2388,  0.7523, -0.7483],
                      [ 0.2555, -0.8896,  1.2533,  1.1000],
                      [-0.9446,  0.9659,  2.3515,  0.8332],
                      [ 0.8201, -0.2388,  0.7523, -0.7483],
                      [ 0.2555, -0.8896,  1.2533,  1.1000],
                      [-0.9446,  0.9659,  2.3515,  0.8332],
                      [ 0.8201, -0.2388,  0.7523, -0.7483],
                      [ 0.2555, -0.8896,  1.2533,  1.1000],
                      [-0.9446,  0.9659,  2.3515,  0.8332],
                      [ 0.8201, -0.2388,  0.7523, -0.7483],
                      [ 0.2555, -0.8896,  1.2533,  1.1000],
                      [-0.9446,  0.9659,  2.3515,  0.8332],
                      [ 0.8201, -0.2388,  0.7523, -0.7483],
                      [ 0.2555, -0.8896,  1.2533,  1.1000],
                      [-0.9446,  0.9659,  2.3515,  0.8332],
                   

In [28]:
%timeit Batch.cat(batches)

149 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
